In [6]:
import calcbench as cb
from IPython.core.display import display, HTML
import json
import itertools
from tqdm import tqdm, tqdm_notebook
import random
import urllib.error
import requests
import sys
from azure.storage.blob import BlockBlobService
import calcbench as cb
blob_service = BlockBlobService(account_name='calcbench',
                            account_key='ct2IzcrN3eX5EfxlSOYqFDbhwZnpIUn2BUQ5ApZz1V6rU0wetgAtqMKR6/BmVlsC8S6F1enlv8UoGtXHHjLPlg==')

In [12]:
cb.api_client._rig_for_testing()

In [8]:
tenKQsections = cb.document_types()['tenKQSections']
disclosure_name_to_disclosure_type = {d['name'] : d['arcrole'] for d in tenKQsections}

In [10]:
periods = [(year, period) for year in range(2010, 2018) for period in [0, 1, 2, 3]]

In [20]:
missing_docs = []
def get_documents_all_companies(disclosure_type, year, period):
    total_length = 0
    try:
        found_documents = list(cb.document_search(entire_universe=True, document_type=disclosure_type, year=year, period=period))
    except requests.RequestException:
        print(disclosure_type, year, period)
    else:
        contents_length = 0
        docs = []
        for  doc in tqdm_notebook(found_documents, desc="{0} {1}-{2}".format(disclosure_type, year, period), leave=False):
            try:
                total_length += blob_service.get_blob_properties(container_name="fact-extract", blob_name=doc['blob_id']).properties.content_length
            except requests.RequestException:
                missing_docs.append(doc)
            except Exception as e:
                print(disclosure_type, year, period, doc['ticker'], e)
    return total_length

In [13]:
test_doc_periods = list(itertools.product(tenKQsections, periods))

In [ ]:
#total_length = 0
for test_doc, period in tqdm_notebook(test_doc_periods[38:], desc="period loop"):
    total_length += get_documents_all_companies(test_doc['arcrole'], year=period[0], period=period[1])
    print(total_length)

1110 2011 2 RZOR The specified blob does not exist.

7790928291


7909551844


8429090255


8541506414


8652284685


...

15120138133


15144693947


15159295972


15175090014


15191790835


In [23]:
log

NameError: name 'log' is not defined

In [24]:
import math

In [28]:
math.log2(15191790835)

33.822572896164985